In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 2397.3116686 ,  3403.89050021],
        [ 2972.13657819,  2526.82859849],
        [ 4391.35938282,  4551.50503312],
        [ 2962.92124502,  3317.68641716],
        [ 3576.30729231,  2730.76993355],
        [ 1907.22712193,  2246.15137891],
        [ 2267.77767399,  3014.72444069],
        [ 3761.01731468,  3171.92466399],
        [ 3961.42761319,  5381.7283612 ],
        [ 5053.84798728,  4242.65635446],
        [ 3000.87543888,  4359.91425011],
        [ 4336.52453263,  4375.72044637],
        [ 4423.21058749,  4356.78145045],
        [ 3250.54600988,  3548.47865437],
        [ 2737.93892372,  5661.60660252],
        [ 4951.87169606,  3162.9181512 ],
        [ 3008.2822909 ,  2376.1562092 ],
        [ 2405.90876275,  3598.51338185],
        [ 4135.52045697,  4119.64028174],
        [ 5238.23178869,  8321.06261477],
        [ 5440.71986222,  3394.63439287],
        [ 2067.80285041,  3054.0533749 ],
        [ 2504.8986245 ,  3564.50743253],
        [ 3438.70962085,  2677.871

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.26879370e+06  0.00000000e+00  4.50e+03 2.42e+02  9.52e+05     0s
   1   1.01989804e+06 -1.76970722e+06  6.74e+02 1.47e+02  1.87e+05     0s
   2   4.89213128e+05 -1.00035885e+06  6.74e-04 5.68e-14  2.66e+04     0s
   3  -4.69741777e+04 -2.56961545e+05  6.73e-10 2.84e-14  3.7

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.6481579793822668e-09,
 3563.8545356149225,
 1.8091343641793238e-09,
 3713.613425340867,
 1428.7397101680099,
 2.6568438500099303e-05,
 5.349807817211262e-07,
 0.0004487906753124172,
 963.0720624257141,
 4120.657315896594,
 1384.3480142276187,
 886.2033500848895,
 2.656839794487007e-05,
 2.6720526225434418e-05,
 5854.874980815523,
 2335.4877889796267,
 6.957504338275463,
 1315.3771286086921,
 5825.22614033648,
 8195.048828964656,
 2.657627349134388e-05,
 1480.7074631142705,
 2534.220319807845,
 175.84310766251994,
 27.06789489612453,
 3102.2620282550374,
 1.52329097736873e-05,
 2756.9968837525917,
 1.397227086233947,
 2.0377954289051274,
 54.97043054912344,
 7294.720088254931,
 3421.8090439485522,
 1.508024169939845e-05,
 1.508788970892627e-05,
 1.7441967288575651e-06,
 1.7441967158195054e-06,
 1.7338077784112907e-06,
 5.704297794967676,
 5889.286264031378,
 5572.989201787567,
 1357.0639675680306,
 3.7082849157719098e-06,
 3.708284554345499e-06,
 15238.08274113147,
 13.75644659966937